# Unit 8: Automation

## Contents

* [Getting Started](#Getting-Started)
* [Generating Keys](#Generating-Keys)
* [Creating a Data Store](#Creating-a-Data-Store)
* [Data Retrieval and Processing](#Data-Retrieval-and-Processing)
* [Automating the Process](#Automating-the-Process)
* [Lab Answers](#Lab-Answers)
* [Next Steps](#Next-Steps)
* [Resources](#Resources)
* [Exercise](#Exercise)

### Lab Questions

[1](#Lab-1), [2](#Lab-2), [3](#Lab-3), [4](#Lab-4), [5](#Lab-5), [6](#Lab-6), [7](#Lab-7)

## Getting Started

In this unit, we'll combine concepts in previous units to collect data from an external source, process it, generate a report, and deliver it.  We'll do this using cloud services, specifically [Amazon Web Services](https://aws.amazon.com/) so an AWS account will be required. The services we'll use qualify for the [free tier](https://aws.amazon.com/free/).

In this example, our report will consist of simple stock portfolio information including shares, price, and total value.

While we'll do some initial setup using the web console, most of the work we'll do with AWS will be done programmatically using Python. To do this, we'll need the [*boto3*](https://aws.amazon.com/sdk-for-python/) library, which can be installed with `pip`.

In [3]:
import sys
!{sys.executable} -m pip install boto3

## Generating Keys

Before we can use boto with our AWS accounts, we need to create an access key and a secret key that will be used to identify our account and the fact that we are authorized to make changes to our accounts.  To generate keys, we'll use the AWS web console.  

Once logged into the AWS console, open the IAM interface by selecting "IAM" in the "Security, Identity & Compliance" section within the services menu.

<figure>
  <img src="./images/08-aws-services.png" alt="aws services menu with iam">	
  <figcaption style="text-align: center; font-weight: bold">Security, Identity &amp; Compliance Services</figcaption>
</figure>

IAM, or Identity and Access Management, allows us to manage access to an AWS account.  IAM allows us to create users and groups with granular permissions.  In multi-user organizations, IAM can be used to ensure that people only have access to services they need and don't have access to other services.  Access control for programs we create can also be managed through IAM.  

Often groups of users will have the same set of permissions.  Rather than repeatedly assigning the same permissions to those users, we instead create a group to which the permissions are assigned and then indicate that users are members of the group.  This provides additional flexibility by allowing different users to be members of different groups.  


To begin, select "Groups" from the left side of the console.

<figure>
  <img src="./images/08-aws-iam-groups.png" alt="iam menu with users selected">	
  <figcaption style="text-align: center; font-weight: bold">IAM Menu</figcaption>
</figure>

To create a new group, click the "Create New Group" button. While it is generally best practice to make permissions as limited as possible, this can be a time consuming process.  For our purposes, we'll create a group with "power user" access - access that allows members the group to create, modify, delete, and manage most of the AWS services. 

For the new group's name, enter a descriptive value that reflects the permissions that will be assigned to the group such as "powerusers". Click "Next Step" to continue.

<figure>
  <img src="./images/08-aws-iam-group-1.png" alt="specify a group name">	
  <figcaption style="text-align: center; font-weight: bold">Specify a Group Name</figcaption>
</figure>

Next, we attach a policy to the group.  Policies define the permissions a group has.  We can filter these by choosing a filter from the drop-down list.  Filter by "Job function".  Among the policy names, we see "PowerUserAccess" - this provides full control to all AWS services with the exception of user management.  Click "Next Step" to continue.

<figure>
  <img src="./images/08-aws-iam-group-2.png" alt="attach a policy">	
  <figcaption style="text-align: center; font-weight: bold">Attach a Policy</figcaption>
</figure>

As a final step, we have a chance to review the settings for the new group.  If everything appears correct, click "Create Group".  Click "Cancel" or "Previous" if changes need to be made.

<figure>
  <img src="./images/08-aws-iam-group-3.png" alt="review group options">
  <figcaption style="text-align: center; font-weight: bold">Review Group Options</figcaption>
</figure>

Now that we have a group, we'll add a user.  On the left, select "Users".  We see the list of existing users. To add a user, click the "Add User" button.  This will begin the user creation process. In the first step, we will specify the user name an the access type.  If we were creating a user for a person, we might choose a user name based on the person's name; because we are creating a user name for a program, we should choose a descriptive name like "report_generator". We'll use this account for programmatic access so select the "Programmatic access" type.  Click "Next: Permissions" to continue.

<figure>
  <img src="./images/08-aws-iam-user-1.png" alt="first step in user creation process">	
  <figcaption style="text-align: center; font-weight: bold">Specify a User Name and Access Type</figcaption>
</figure>

To assign permissions to a user, we specify the groups to which the user will belong.  For our purposes, the new user will be a member of the group we created; select the appropriate group.  Click "Next: Review" to continue.

<figure>
  <img src="./images/08-aws-iam-user-2.png" alt="second step in user creation process">
  <figcaption style="text-align: center; font-weight: bold">Specify Group Membership</figcaption>
</figure>

Next, we have a chance to review the settings for the new user.  If everything appears correct, click "Create User".  Click "Cancel" or "Previous" if changes need to be made.

<figure>
  <img src="./images/08-aws-iam-user-3.png" alt="third step in user creation process">
  <figcaption style="text-align: center; font-weight: bold">Review User Options</figcaption>
</figure>

Finally, we see a success message and the access key for the new user.  We can also choose to see the secret access key.  Enter these values in the cell below.

<figure>
  <img src="./images/08-aws-iam-user-4.png" alt="final step in user creation process">
  <figcaption style="text-align: center; font-weight: bold">User Access and Secret Keys</figcaption>
</figure>

<hr>
<a name="Lab-1"></a><mark> **Lab 1** In the cell below, set the values for `ACCESS_KEY` and `SECRET_KEY` using the values displayed in the AWS console.
</mark>

In [154]:
ACCESS_KEY = ""
SECRET_KEY = ""

<hr>

## Creating a Data Store

Our objective is to automate data collection and processing and report generation and delivery.  Once we've collected the data and completed some initial processing, we'll likely need to store it for future use.  While we could create a relational database using the [Amazon Relational Database Service](https://aws.amazon.com/rds/), we'll instead rely on a NoSQL store using [Amazon SimpleDB](https://aws.amazon.com/simpledb/).

To work with SimpleDB from boto, we must first create a [*client()*](https://boto3.readthedocs.io/en/latest/guide/quickstart.html) indicating the service we intend to use and with our credentials.

For this example, we'll use the "us-east-1" region as it includes all the AWS services we need.

In [130]:
import boto3
sdb_client = boto3.client('sdb',
                          aws_access_key_id = ACCESS_KEY,
                          aws_secret_access_key = SECRET_KEY,
                          region_name="us-east-1")

Instead of databases and tables, SimpleDB uses *domains* to categorize data.  To create a new domain, we can use the client's [*create_domain()*](https://boto3.readthedocs.io/en/latest/reference/services/sdb.html#SimpleDB.Client.create_domain) method.

In [131]:
sdb_client.create_domain(DomainName="test_domain")

{'ResponseMetadata': {'BoxUsage': '0.0055590278',
  'HTTPHeaders': {'connection': 'keep-alive',
   'content-type': 'text/xml',
   'date': 'Sun, 22 Apr 2018 18:00:56 GMT',
   'server': 'Amazon SimpleDB',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '65344e29-625b-708a-f290-9b3856a5e21c',
  'RetryAttempts': 0}}

If successful, the *create_domain()* method returns information related to the domain.  Note that an HTTP status code of [200](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes#2xx_Success) indicates success.  We can also verify that the domain was created successfully by viewing its metadata.  To do this, we can use the client's [*domain_metadata()*](https://boto3.readthedocs.io/en/latest/reference/services/sdb.html#SimpleDB.Client.domain_metadata) method.  When calling the method, we specify the domain name using the `DomainName` keyword argument.

<hr>
<a name="Lab-2"></a><mark> **Lab 2** In the cell below, retrieve the metadata for the recently-created domain using the *domain_metadata_method()*.
</mark>

In [132]:
sdb_client.domain_metadata(DomainName='test_domain')

{'AttributeNameCount': 0,
 'AttributeNamesSizeBytes': 0,
 'AttributeValueCount': 0,
 'AttributeValuesSizeBytes': 0,
 'ItemCount': 0,
 'ItemNamesSizeBytes': 0,
 'ResponseMetadata': {'BoxUsage': '0.0000071759',
  'HTTPHeaders': {'connection': 'keep-alive',
   'content-type': 'text/xml',
   'date': 'Sun, 22 Apr 2018 18:01:02 GMT',
   'server': 'Amazon SimpleDB',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '2c593445-06e5-965d-6b1c-5315659983e3',
  'RetryAttempts': 0},
 'Timestamp': 1524420062}

<hr>

With a domain created, we can now begin adding data; to do this, we can use the [*put_attributes()*](https://boto3.readthedocs.io/en/latest/reference/services/sdb.html#SimpleDB.Client.put_attributes) method.  Data in SimpleDB is stored as items where each item can have a set of attributes associated with it.  In the code below, we add an item named `employee1` to represent an employee.  In the example, there are two attributes associated with the employee, full name and department.  To associate these with the item, we specify a list of dictionaries for attributes where each dictionary has two key/value pairs.  Each attribute dictionary contains a `Name` and `Value` key where the corresponding values are the data for each attribute. If we think of our data as being tabular, we can think of the `ItemName` as identifying a row, the attribute `Name` key as specifying a column name, and the attribute `Value` key as being used to provide a field value.  

Given data that could be stored like this

<table>
    <tr>
        <th>ID</th>
        <th>Fullname</th>
        <th>Department</th>
    </tr>
    <tr>
        <td>employee1</td>
        <td>Bob Smith</td>
        <td>Marketing</td>
    </tr>
</table>

we can store it in SimpleDB using the following code.

In [133]:
sdb_client.put_attributes(DomainName='test_domain',
                          ItemName='employee1',
                          Attributes=[
                              {
                                  'Name': 'Fullname',
                                  'Value': 'Bob Smith'
                              },
                              {
                                  'Name': 'Department',
                                  'Value': 'Marketing'
                              }
                          ])

{'ResponseMetadata': {'BoxUsage': '0.0000219923',
  'HTTPHeaders': {'connection': 'keep-alive',
   'content-type': 'text/xml',
   'date': 'Sun, 22 Apr 2018 18:01:26 GMT',
   'server': 'Amazon SimpleDB',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '1de68219-3132-37d2-8bb6-df8742469246',
  'RetryAttempts': 0}}

<hr>
<a name="Lab-3"></a><mark> **Lab 3** In the cell below, use the *put_attributes()* method to add data for the following item.
    
<table>
    <tr>
        <th>ID</th>
        <th>Fullname</th>
        <th>Department</th>
    </tr>
    <tr>
        <td>employee2</td>
        <td>Jane Doe</td>
        <td>IT</td>
    </tr>
</table>

</mark>

In [134]:
sdb_client.put_attributes(DomainName='test_domain',
                          ItemName='employee2',
                          Attributes=[
                              {
                                  'Name': 'Fullname',
                                  'Value': 'Jane Doe'
                              },
                              {
                                  'Name': 'Department',
                                  'Value': 'IT'
                              }
                          ])

{'ResponseMetadata': {'BoxUsage': '0.0000219923',
  'HTTPHeaders': {'connection': 'keep-alive',
   'content-type': 'text/xml',
   'date': 'Sun, 22 Apr 2018 18:02:22 GMT',
   'server': 'Amazon SimpleDB',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': 'e6061205-37aa-15b4-de8b-ef7dd8c394a6',
  'RetryAttempts': 0}}

<hr>

To retrieve data for a specific item, we can use the [*get_attributes()*](https://boto3.readthedocs.io/en/latest/reference/services/sdb.html#SimpleDB.Client.get_attributes) method where we specify the item's name. Note that the returned value is a dictionary containing both item attributes and metadata.

In [135]:
sdb_client.get_attributes(DomainName='test_domain', ItemName="employee1")

{'Attributes': [{'Name': 'Department', 'Value': 'Marketing'},
  {'Name': 'Fullname', 'Value': 'Bob Smith'}],
 'ResponseMetadata': {'BoxUsage': '0.0000093282',
  'HTTPHeaders': {'connection': 'keep-alive',
   'content-type': 'text/xml',
   'date': 'Sun, 22 Apr 2018 18:02:24 GMT',
   'server': 'Amazon SimpleDB',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '66888e0f-c247-9f9b-01a4-40103608c3b2',
  'RetryAttempts': 0}}

To get a specific attribute for a specific item, we can specify the attribute name in a list with the `AttributeNames` keyword argument.

In [136]:
sdb_client.get_attributes(DomainName='test_domain', ItemName="employee1", AttributeNames=['Department'])

{'Attributes': [{'Name': 'Department', 'Value': 'Marketing'}],
 'ResponseMetadata': {'BoxUsage': '0.0000093222',
  'HTTPHeaders': {'connection': 'keep-alive',
   'content-type': 'text/xml',
   'date': 'Sun, 22 Apr 2018 18:02:25 GMT',
   'server': 'Amazon SimpleDB',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': 'be126a52-adfb-7f18-19ff-8a61467e7cee',
  'RetryAttempts': 0}}

We can also use the [*select()*](https://boto3.readthedocs.io/en/latest/reference/services/sdb.html#SimpleDB.Client.select) method to find data using SQL-like queries.  For example, we can use the following code to find all employees in the marketing department.  Notice that the domain name is used like a table name in the query.

In [137]:
sdb_client.select(SelectExpression="select * from test_domain where Department = 'Marketing'")

{'Items': [{'Attributes': [{'Name': 'Department', 'Value': 'Marketing'},
    {'Name': 'Fullname', 'Value': 'Bob Smith'}],
   'Name': 'employee1'}],
 'ResponseMetadata': {'BoxUsage': '0.0000228616',
  'HTTPHeaders': {'connection': 'keep-alive',
   'content-type': 'text/xml',
   'date': 'Sun, 22 Apr 2018 18:02:31 GMT',
   'server': 'Amazon SimpleDB',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': 'a34a3876-80a9-2ddd-02da-9538b7a6aee3',
  'RetryAttempts': 0}}

<hr>
<a name="Lab-4"></a><mark> **Lab 4** In the cell below, use the *select()* method to find data where the employee's full name is "Bob Smith".

In [138]:
sdb_client.select(SelectExpression="select * from test_domain where Fullname = 'Bob Smith'")

{'Items': [{'Attributes': [{'Name': 'Department', 'Value': 'Marketing'},
    {'Name': 'Fullname', 'Value': 'Bob Smith'}],
   'Name': 'employee1'}],
 'ResponseMetadata': {'BoxUsage': '0.0000228616',
  'HTTPHeaders': {'connection': 'keep-alive',
   'content-type': 'text/xml',
   'date': 'Sun, 22 Apr 2018 18:02:58 GMT',
   'server': 'Amazon SimpleDB',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': 'c8de410e-2aac-efe2-79ce-ba894dcf1197',
  'RetryAttempts': 0}}

<hr>

We can update attributes for records using the *put_attributes()* method.  To do this, we specify the appropriate attribute `Name` and `Value` keys/value pairs along with `Replace: True`.  For example, we can change `employee1`'s name.

In [139]:
sdb_client.put_attributes(DomainName='test_domain',
                          ItemName='employee1',
                          Attributes=[
                              {
                                  'Name': 'Fullname',
                                  'Value': 'Robert Smith',
                                  'Replace': True
                              }
                          ])

{'ResponseMetadata': {'BoxUsage': '0.0000219909',
  'HTTPHeaders': {'connection': 'keep-alive',
   'content-type': 'text/xml',
   'date': 'Sun, 22 Apr 2018 18:02:59 GMT',
   'server': 'Amazon SimpleDB',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '70e97883-0557-5e79-6d3e-7c6ff3a7da6b',
  'RetryAttempts': 0}}

We can confirm the change using the *get_attributes()* method.

In [140]:
sdb_client.get_attributes(DomainName='test_domain', ItemName="employee1")

{'Attributes': [{'Name': 'Department', 'Value': 'Marketing'},
  {'Name': 'Fullname', 'Value': 'Robert Smith'}],
 'ResponseMetadata': {'BoxUsage': '0.0000093282',
  'HTTPHeaders': {'connection': 'keep-alive',
   'content-type': 'text/xml',
   'date': 'Sun, 22 Apr 2018 18:03:02 GMT',
   'server': 'Amazon SimpleDB',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': 'fff0890f-a234-2673-ef60-9c2ef4d822fa',
  'RetryAttempts': 0}}

We can delete items using the [*delete_attributes()*] method, specifying the domain and item names.

In [141]:
sdb_client.delete_attributes(DomainName='test_domain', ItemName="employee1")

{'ResponseMetadata': {'BoxUsage': '0.0000219907',
  'HTTPHeaders': {'connection': 'keep-alive',
   'content-type': 'text/xml',
   'date': 'Sun, 22 Apr 2018 18:03:05 GMT',
   'server': 'Amazon SimpleDB',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': 'd3f2cb21-41ef-d5fb-3e7a-fc7843e684fc',
  'RetryAttempts': 0}}

<hr>
<a name="Lab-5"></a><mark> **Lab 5** In the cell below, use the *delete_attributes()* method to delete the `employee2`item.

In [142]:
sdb_client.delete_attributes(DomainName='test_domain', ItemName="employee2")

{'ResponseMetadata': {'BoxUsage': '0.0000219907',
  'HTTPHeaders': {'connection': 'keep-alive',
   'content-type': 'text/xml',
   'date': 'Sun, 22 Apr 2018 18:03:27 GMT',
   'server': 'Amazon SimpleDB',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '51e43e34-6c25-d959-f487-35b262a98347',
  'RetryAttempts': 0}}

<hr>

At this point, our domain should be empty.  We can confirm this using the *domain_metadata()* method.

In [143]:
sdb_client.domain_metadata(DomainName='test_domain')

{'AttributeNameCount': 0,
 'AttributeNamesSizeBytes': 0,
 'AttributeValueCount': 0,
 'AttributeValuesSizeBytes': 0,
 'ItemCount': 0,
 'ItemNamesSizeBytes': 0,
 'ResponseMetadata': {'BoxUsage': '0.0000071759',
  'HTTPHeaders': {'connection': 'keep-alive',
   'content-type': 'text/xml',
   'date': 'Sun, 22 Apr 2018 18:03:28 GMT',
   'server': 'Amazon SimpleDB',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '2f0ec258-3f47-91eb-e656-9c619cfd87d1',
  'RetryAttempts': 0},
 'Timestamp': 1524420208}

To delete the domain entirely, we can use the [*delete_domain()*](https://boto3.readthedocs.io/en/latest/reference/services/sdb.html#SimpleDB.Client.delete_domain) method.

In [144]:
sdb_client.delete_domain(DomainName='test_domain')

{'ResponseMetadata': {'BoxUsage': '0.0055590278',
  'HTTPHeaders': {'connection': 'keep-alive',
   'content-type': 'text/xml',
   'date': 'Sun, 22 Apr 2018 18:03:32 GMT',
   'server': 'Amazon SimpleDB',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '56b81611-5479-d1f7-a55a-cbfcf4b97778',
  'RetryAttempts': 0}}

SimpleDB provides finer control over the data such as deleting individual attributes rather than entire items. The methods we've looked at will be sufficient for our task.

Now that we have a way of storing data, we need a way of obtaining the data and processing it.

## Data Retrieval and Processing

Suppose we had a simple portfolio consisting of shares of `goog` and `appl`.  Our report generation process would need to retrieve prices for each.  To do this, use the [IEX API](https://iextrading.com/developer/docs/#getting-started). For our simple report, we'll use only the latest price data accessible using the [price](https://iextrading.com/developer/docs/#price) endpoint.  

To retrieve and process the data into Python objects, we'll use the [Requests](http://docs.python-requests.org/en/master/) library.  In the example below, we get the latest price for `goog`.  Recall that the response contain JSON data that can be processed using the *json()* method.

In [145]:
import requests

response = requests.get("https://api.iextrading.com/1.0/stock/goog/price")
price = response.json()
price

1072.96

We can store price data and number of shares for each stock in a SimpleDB domain.  To start, we'll create the domain and an item for each stock where the stock's name will be used as the item name. Price and number of shares will be stored as attributes.

In [146]:
sdb_client.create_domain(DomainName='portfolio')

for name in ['aapl', 'goog']:
    sdb_client.put_attributes(DomainName='portfolio',
                              ItemName=name,
                              Attributes=[
                                  {
                                      'Name': 'price',
                                      'Value': '0'
                                  },
                                  {
                                      'Name': 'shares',
                                      'Value': '5'
                                  }
                              ])

We can confirm that the items existing using the *select()* method.

In [147]:
sdb_client.select(SelectExpression="select * from portfolio")

{'Items': [{'Attributes': [{'Name': 'shares', 'Value': '5'},
    {'Name': 'price', 'Value': '165.72'},
    {'Name': 'price', 'Value': '0'}],
   'Name': 'aapl'},
  {'Attributes': [{'Name': 'shares', 'Value': '5'},
    {'Name': 'price', 'Value': '1072.96'},
    {'Name': 'price', 'Value': '0'}],
   'Name': 'goog'}],
 'ResponseMetadata': {'BoxUsage': '0.0000320033',
  'HTTPHeaders': {'connection': 'keep-alive',
   'content-type': 'text/xml',
   'date': 'Sun, 22 Apr 2018 18:03:44 GMT',
   'server': 'Amazon SimpleDB',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '3207ece2-0428-4b2a-3c8b-2d1516b86224',
  'RetryAttempts': 0}}

We can now write a function that use the IEX API to update the price data in the SimpleDB domain.

In [148]:
def update_portfolio(sdb_client):
    results = sdb_client.select(SelectExpression="select * from portfolio")
    for item in results['Items']:
        name = item['Name']
        url = "https://api.iextrading.com/1.0/stock/{0}/price".format(name)
        price = requests.get(url).json()
        sdb_client.put_attributes(
            DomainName='portfolio',
            ItemName=name,
            Attributes=[
                {
                    'Name': 'price',
                    'Value': str(price),
                    'Replace': True
                }
        ])

Note that we values in SimpleDB are stored as strings so we have to convert the price data to a string when storing it.  

Let's execute the function.

In [149]:
update_portfolio(sdb_client)

To see if the domain was successfully updated, we can use the *select()* method.

In [150]:
sdb_client.select(SelectExpression="select * from portfolio")['Items']

[{'Attributes': [{'Name': 'shares', 'Value': '5'},
   {'Name': 'price', 'Value': '165.72'}],
  'Name': 'aapl'},
 {'Attributes': [{'Name': 'shares', 'Value': '5'},
   {'Name': 'price', 'Value': '1072.96'}],
  'Name': 'goog'}]

Now that we have a way of retrieving the data and updating the SimpleDB domain, we work on reporting with the data.  For this example, our report will be a simple text summary that we email to ourselves. 

In order to generate and send the email, we will use the [AWS Simple Email Service](https://aws.amazon.com/ses/).  In order to use it, we have to first verify an email address.

Log into the AWS console and select "Simple Email Service" from the "Customer Engagement" section.  Make sure the "US East (N. Virginia)" region is selected in the upper right corner of the console.

<figure>
  <img src="./images/08-aws-ses-1.png" alt="aws services menu with ses">	
  <figcaption style="text-align: center; font-weight: bold">Customer Engagement Services</figcaption>
</figure>

On the left, click the "Email Adresses" link in the "Identity Management" section.

<figure>
  <img src="./images/08-aws-ses-2.png" alt="ses email addresses">	
  <figcaption style="text-align: center; font-weight: bold">SES Identity Management</figcaption>
</figure>

In order to send an email, we'll need to provide a source address. In order to use that address, we have to verify that we own/control it.  Similarly, we'll have to verify that the destination address is willing to receive email.  To simplify things, we'll use the same email address as the sender and recipient. To begin the verification process, click "Verify a New Email Address". 

<figure>
  <img src="./images/08-aws-ses-3.png" alt="ses verify a new email address button">	
  <figcaption style="text-align: center; font-weight: bold">Verify a New Email Address</figcaption>
</figure>

In the pop-up, enter your email address and click the "Verify This Email Address Button".

<figure>
  <img src="./images/08-aws-ses-4.png" alt="email address pop-up">	
  <figcaption style="text-align: center; font-weight: bold">Enter an Email Address</figcaption>
</figure>

Check your email account - you should receive an email with a link to verify the address.  Click it to complete the verification process. 

To send an email, we'll need to create a client using Boto3 in much the same way we did for SimpleDB.

In [151]:
ses_client = boto3.client('ses',
                          aws_access_key_id = ACCESS_KEY,
                          aws_secret_access_key = SECRET_KEY,
                          region_name="us-east-1")

To send an email, we can use the [*send_email()*](https://boto3.readthedocs.io/en/latest/reference/services/ses.html#SES.Client.send_email) method, specifying the source and destination addresses as well as the message.  

<hr>
<a name="Lab-6"></a><mark> **Lab 6** Update the code below to store your email address in the `ADDRESS` variable.  After executing the cell, you should receive a test message.

In [101]:
ADDRESS = ""
CHARSET = "UTF-8"

ses_client.send_email(
    Source=ADDRESS,
    Destination={
        'ToAddresses': [
            ADDRESS
        ]
    },
    Message={
        'Body': {
            'Text': {
                'Charset': CHARSET,
                'Data': "Testing SES"
            }
        },
        'Subject': {
            'Charset': CHARSET,
            'Data': "This is a test"
        }
    }
)

{'MessageId': '01000162ee28281f-862b375c-74e4-4ee6-903b-93b948b3e61d-000000',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '326',
   'content-type': 'text/xml',
   'date': 'Sun, 22 Apr 2018 16:20:31 GMT',
   'x-amzn-requestid': '141f6a57-4649-11e8-9b8a-3bd3697c786c'},
  'HTTPStatusCode': 200,
  'RequestId': '141f6a57-4649-11e8-9b8a-3bd3697c786c',
  'RetryAttempts': 0}}

<hr>

We can now combine the ability to retrieve data from SimpleDB with our ability to send email using the Simple Email Service, to create and send a report.  To do this, we'll create two functions.  The first will generate the report and the second will send the report email.

In [152]:
def report(sdb_client):
    email_lines = []
    total = 0 
    
    results = sdb_client.select(SelectExpression="select * from portfolio")
    for item in results['Items']:
        name = item['Name']
        shares = int(sdb_client.get_attributes(DomainName='portfolio', 
                                               ItemName=name, 
                                               AttributeNames=['shares'])['Attributes'][0]['Value'])
        price = float(sdb_client.get_attributes(DomainName='portfolio', 
                                                ItemName=name, 
                                                AttributeNames=['price'])['Attributes'][0]['Value'])
        stock_total = shares * price
        message = f"{name}, {shares} shares @ ${price}: ${stock_total}"
        email_lines.append(message)
        total += stock_total
        
    email_lines.append(f"Total: ${total}")
    return "\n".join(email_lines)

print(report(sdb_client))

aapl, 5 shares @ $165.72: $828.6
goog, 5 shares @ $1072.96: $5364.8
Total: $6193.400000000001


To send the email, we have the following.

In [153]:
def send_report(ses_client, dest_addr, message):
    CHARSET = "UTF-8"

    ses_client.send_email(
        Source=dest_addr,
        Destination={
            'ToAddresses': [
                dest_addr
            ]
        },
        Message={
            'Body': {
                'Text': {
                    'Charset': CHARSET,
                    'Data': message
                }
            },
            'Subject': {
                'Charset': CHARSET,
                'Data': "Report"
            }
        }
    )

We can now test report generation and message delivery.

<hr>
<a name="Lab-7"></a><mark> **Lab 7** Update the code below with the appropriate email address to test report generation and delivery.

In [127]:
ADDRESS = ""
message = report(sdb_client)
send_report(ses_client, ADDRESS, message)

<hr>

## Automating the Process

While we could manually execute code when necessary, we can improve reliability by automating as much of the process as possible.  There are a variety of way of automating execution including creating [scheduled tasks in Windows](https://msdn.microsoft.com/en-us/library/windows/desktop/aa383614%28v=vs.85%29.aspx) and using [cron](https://en.wikipedia.org/wiki/Cron) in Unix-like environments.  For our reporting job, we'll use [AWS Lambda](https://aws.amazon.com/lambda/), a service that allows us to execute code in the cloud without having to worry about managing servers.  

While we could programmatically set up Lambda using Boto, it is much simpler to to it via the console.  From the Services menu, select "Lambda" in the "Compute" section.  

<figure>
  <img src="./images/08-aws-lambda-1.png" alt="Lambda in the compute section">	
  <figcaption style="text-align: center; font-weight: bold">Compute Services</figcaption>
</figure>

When the Lambda page loads, make sure that your region is set to "US East (N. Virginia)". Click the "Create a function" button to get started. We'll author a function from scratch.  Set the following values for the fields and click "Create function".

- Name: report
- Runtime: Python 3.6
- Role: Create new role from template(s)
- Role name: report
- Policy templates: Basic Edge Lambda permissions

<figure>
  <img src="./images/08-aws-lambda-2.png" alt="new function properties">	
  <figcaption style="text-align: center; font-weight: bold">New Lambda Function</figcaption>
</figure>

There are several sections of configuration items; we'll look at a few of them.  First, look for the "Environment Variables" section.  We can used environment variables to provide data to our function when it runs while avoiding hard-coding the data in the function itself.  For example. we can store the AWS keys and the email address that the report will be sent to as environment variables.  Create the following three keys and enter the appropriate values.

- ACCESS_KEY 
- SECRET_KEY 
- EMAIL_ADDRESS

<figure>
  <img src="./images/08-aws-lambda-3.png" alt="environment variables">	
  <figcaption style="text-align: center; font-weight: bold">Environment Variables</figcaption>
</figure>

We'll now add code for our function using the code editor in the "Function code" section.  Our code will be based on the *update_portfolio()*, *report()*, and *send_report()* functions we wrote earlier.


``` python
import os
import boto3
from botocore.vendored import requests


def update_portfolio(sdb_client):
    results = sdb_client.select(SelectExpression="select * from portfolio")
    for item in results['Items']:
        name = item['Name']
        url = "https://api.iextrading.com/1.0/stock/{0}/price".format(name)
        price = requests.get(url).json()
        sdb_client.put_attributes(
            DomainName='portfolio',
            ItemName=name,
            Attributes=[
                {
                    'Name': 'price',
                    'Value': str(price),
                    'Replace': True
                }
        ])
        

def report(sdb_client):
    email_lines = []
    total = 0 
    
    results = sdb_client.select(SelectExpression="select * from portfolio")
    for item in results['Items']:
        name = item['Name']
        shares = int(sdb_client.get_attributes(DomainName='portfolio', 
                                               ItemName=name, 
                                               AttributeNames=['shares'])['Attributes'][0]['Value'])
        price = float(sdb_client.get_attributes(DomainName='portfolio', 
                                                ItemName=name, 
                                                AttributeNames=['price'])['Attributes'][0]['Value'])
        stock_total = shares * price
        message = f"{name}, {shares} shares @ ${price}: ${stock_total}"
        email_lines.append(message)
        total += stock_total
        
    email_lines.append(f"Total: ${total}")
    return "\n".join(email_lines)
    
    
def send_report(ses_client, dest_addr, message):
    CHARSET = "UTF-8"

    ses_client.send_email(
        Source=dest_addr,
        Destination={
            'ToAddresses': [
                dest_addr
            ]
        },
        Message={
            'Body': {
                'Text': {
                    'Charset': CHARSET,
                    'Data': message
                }
            },
            'Subject': {
                'Charset': CHARSET,
                'Data': "Report"
            }
        }
    )    


def lambda_handler(event, context):
    ACCESS_KEY = os.environ['ACCESS_KEY']
    SECRET_KEY = os.environ['SECRET_KEY']
    EMAIL_ADDRESS = os.environ['EMAIL_ADDRESS']
    
    sdb_client = boto3.client('sdb',
                              aws_access_key_id = ACCESS_KEY,
                              aws_secret_access_key = SECRET_KEY,
                              region_name="us-east-1")
                              
    ses_client = boto3.client('ses',
                              aws_access_key_id = ACCESS_KEY,
                              aws_secret_access_key = SECRET_KEY,
                              region_name="us-east-1")

    update_portfolio(sdb_client)
    message = report(sdb_client)
    send_report(ses_client, EMAIL_ADDRESS, message)
```

The *lambda_handler()* function serves as the entry point for the code.  We start by loading data from the environment variables we set previously.  Next, we create the Boto3 clients.  Finally, we execute the functions we created earlier in order.  

To test the code, first click "Save" then click "Test" in the upper right corner of the console. This will prompt us to define a new test event; we can create a new test event with the with no initial data.  After defining the test event, click "Test" again.

<figure>
  <img src="./images/08-aws-lambda-4.png" alt="test event">	
  <figcaption style="text-align: center; font-weight: bold">New Test Event</figcaption>
</figure>

If everything is configured properly, you should see a success message in the AWS console and receive a report email.

Finally, to automate the function, we need to schedule its execution.  In the "Designer" select "CloudWatch Events". Below report, a "CloudWatch Events" item appears and is selected - we can navigate back to the report function to configure its properties if necessary.  

<figure>
  <img src="./images/08-aws-lambda-5.png" alt="cloudwatch">	
  <figcaption style="text-align: center; font-weight: bold">CloudWatch Events</figcaption>
</figure>

In the "Configure triggers' section, we can create a new rule based on a scheduled expression to run the Lambda function as frequently as we'd like; the image below shows the configuration for daily execution. See the [AWS Lambda documentation](https://docs.aws.amazon.com/lambda/latest/dg/tutorial-scheduled-events-schedule-expressions.html) for details regarding valid expressions.

<figure>
  <img src="./images/08-aws-lambda-6.png" alt="cloudwatch triggers">	
  <figcaption style="text-align: center; font-weight: bold">CloudWatch Event Trigger</figcaption>
</figure>

## Lab Answers

1. Values vary

2. ```python
   sdb_client.domain_metadata(DomainName='test_domain')
   ```
   
3. ```python
   sdb_client.put_attributes(DomainName='test_domain',
                             ItemName='employee2',
                             Attributes=[
                                 {
                                     'Name': 'Fullname',
                                     'Value': 'Jane Doe'
                                 },
                                 {
                                     'Name': 'Department',
                                     'Value': 'IT'
                                 }
                             ])
   ```
   
4. ```python
   sdb_client.select(SelectExpression="select * from test_domain where Fullname = 'Bob Smith'")
   ```
   
5. ```python
   sdb_client.delete_attributes(DomainName='test_domain', ItemName="employee2")
   ```
   
6. Values vary
   
7. Values vary

## Next Steps

Automation should be a goal for most of the tasks we undertake - even complicated ones.  While we've only looked at automating a simple task of retrieving text data, creating a text report, and emailing the report as plain text, it is possible to automate more complicated analytics tasks, generate a variety of outputs, and to use the outputs to create more than just emails - we could update dashboards, for example.  Continue to explore different data sources and think of what information could be extracted from the data and how that information could be conveyed to interested parties.  Try to automate these processes.

## Resources

- [Automate the Boring Stuff with Python by Al Sweigart](https://automatetheboringstuff.com/)
- [Azure for Python Developers](https://docs.microsoft.com/en-us/python/azure/?view=azure-python)
- [Boto3 Code Examples](https://boto3.readthedocs.io/en/latest/guide/examples.html)
- [Google Cloud Platform: Using the Python Client Library](https://cloud.google.com/compute/docs/tutorials/python-guide)

## Exercise

Modify the reporting functions to store and report on several days worth of stock data rather than just the latest price data.  Test your code.